<a href="https://colab.research.google.com/github/mgonzalz/ppd_regresion-distribuida/blob/main/regresion_distribuida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **Regresión con PySpark.**
Mediante las técnicas de programación paralela y distribuida que ofrece PySpark, debemos de determinar el índice de Rendimiento Académico (*Performance Index*) que tienen los estudiantes atendiendo a una serie de variables.

El dataset del que partimos tiene las siguientes variables independientes:

* Hours Studied: El número total de horas que ha estudiado cada estudiante.
* Previous Scores: Las notas que ha obtenido un estudiante en exámenes anteriores.
* Extracurricular Activities: Si el estudiante participa en actividades extraescolares (Yes or No).
* Sleep Hours: El número de horas que un estudiante duerme al día.
* Sample Question Papers Practiced: El número de hojas de ejercicios que el estudiante realizó.
Se trata de predecir el valor de la variable dependiente:

* Performance Index: La medida del rendimiento académico de cada estudiante. Los valores están comprendidos entre 10 y 100, indicando los valores más altos un mayor rendimiento académico.

Se valuará el modelo mediante **Root Mean Squared Error**.

## Importaciones e instalación.

Para evitar la subida de archivos manualmente en Google Colab, clonaremos directamente el repositorio donde se encuentran alojados estos archivos y luego los añadiremos al directorio actual.







In [1]:
!git clone https://github.com/mgonzalz/ppd_regresion-distribuida.git

Cloning into 'ppd_regresion-distribuida'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 2), reused 10 (delta 1), pack-reused 0
Receiving objects: 100% (14/14), 49.46 KiB | 538.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
!mv ppd_regresion-distribuida/* ./ # Mover los archivos al directorio general

In [3]:
!rm -rf ppd_regresion-distribuida # Eliminación de la carpeta vacía

El archivo `requirements.txt` contiene todas las versiones de los paquetes utilizados. A través de la siguiente secuencia de código los instalaremos e importaremos los métodos necesarios.

In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f872618adddd1717a485c074846f0f996dbc3b9e01071763b35d5186a1d60447
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [5]:
# Python estándar
import os
import sys
from typing import List

# Configuración de Spark
import findspark
import pyspark
from pyspark.sql import DataFrame, SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

# Modelado y evaluación de Spark ML
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

## Creación del entorno.

Se preparará el entorno para el uso de Spark, descargando este y configurando las variables de entorno necesarias. Mediante `findspark` se habilitará la accesibilidad de Spark desde el entorno de ejecución. Finalmente, se crea la sesión de Spark.

In [6]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

findspark.init()
findspark.find()

spark= SparkSession \
       .builder \
       .appName("regresion") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [783 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,081 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ub

In [7]:
spark=SparkSession.builder.appName('regresion').getOrCreate()

## Estudio del rendimiento académico.

### Importación del dataset.

In [8]:
data = spark.read.csv("/content/data/Student_Performance.csv", header=True, inferSchema=True)
data.show(5)

+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|Hours Studied|Previous Scores|Extracurricular Activities|Sleep Hours|Sample Question Papers Practiced|Performance Index|
+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|            7|             99|                       Yes|          9|                               1|             91.0|
|            4|             82|                        No|          4|                               2|             65.0|
|            8|             51|                       Yes|          7|                               2|             45.0|
|            5|             52|                       Yes|          5|                               2|             36.0|
|            7|             75|                        No|          8|                               5|             66.0|
+-------------+---------

### Preparación del dataset.
Antes del ánalisis de datos, debemos de preparar los datos y tener un conocimiento sobre estos.

In [9]:
data.toPandas().isnull().sum() # Valores nulos.

Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64

In [10]:
data.printSchema()

root
 |-- Hours Studied: integer (nullable = true)
 |-- Previous Scores: integer (nullable = true)
 |-- Extracurricular Activities: string (nullable = true)
 |-- Sleep Hours: integer (nullable = true)
 |-- Sample Question Papers Practiced: integer (nullable = true)
 |-- Performance Index: double (nullable = true)



In [11]:
data.dtypes

[('Hours Studied', 'int'),
 ('Previous Scores', 'int'),
 ('Extracurricular Activities', 'string'),
 ('Sleep Hours', 'int'),
 ('Sample Question Papers Practiced', 'int'),
 ('Performance Index', 'double')]

Miramos la existencia de variables categóricas. Observamos que solo la columna `Extracurricular Activities` contiene de estos y realizaremos la codificación ordinal o *Label Encoding* de estos valores para poder realizar un análisis.





In [12]:
data.select('Extracurricular Activities').distinct().show()

+--------------------------+
|Extracurricular Activities|
+--------------------------+
|                        No|
|                       Yes|
+--------------------------+



En este caso al solo haber dos valores se puede hacer la codificación de manera manual.

In [13]:
data = data.withColumn('Extracurricular Activities', F.when(data["Extracurricular Activities"] == "Yes", 1).otherwise(0))

In [14]:
data.show(5)

+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|Hours Studied|Previous Scores|Extracurricular Activities|Sleep Hours|Sample Question Papers Practiced|Performance Index|
+-------------+---------------+--------------------------+-----------+--------------------------------+-----------------+
|            7|             99|                         1|          9|                               1|             91.0|
|            4|             82|                         0|          4|                               2|             65.0|
|            8|             51|                         1|          7|                               2|             45.0|
|            5|             52|                         1|          5|                               2|             36.0|
|            7|             75|                         0|          8|                               5|             66.0|
+-------------+---------

Consideramos que el dataset ya esta preparado.

### Regresión lineal.

In [15]:
feature_cols = [col for col in data.columns if col != 'Performance Index'] #identificación variables independientes

In [16]:
assembler = VectorAssembler(
    inputCols= feature_cols,
    outputCol="features")

data = assembler.transform(data)
final_data = data.select("features", "Performance Index")

train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=53)

In [17]:
lr = LinearRegression(featuresCol="features", labelCol="Performance Index", predictionCol="y_pred")
lr_model = lr.fit(train_data)

## Evaluación del modelo.

In [18]:
predictions = lr_model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="Performance Index", predictionCol="y_pred", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))

Root Mean Squared Error (RMSE) on test data: 2.068


In [19]:
predictions.show()

+--------------------+-----------------+------------------+
|            features|Performance Index|            y_pred|
+--------------------+-----------------+------------------+
|[1.0,40.0,0.0,4.0...|             15.0|11.993651539290596|
|[1.0,40.0,0.0,5.0...|             10.0| 13.64685396290519|
|[1.0,40.0,0.0,9.0...|             11.0|14.182652422722384|
|[1.0,40.0,1.0,7.0...|             12.0|14.618491772149618|
|[1.0,41.0,0.0,5.0...|             14.0|12.901483761986846|
|[1.0,41.0,1.0,8.0...|             13.0|14.938226133428984|
|[1.0,42.0,1.0,5.0...|             15.0|15.506292516780334|
|[1.0,42.0,1.0,9.0...|             17.0|  17.8063845608482|
|[1.0,42.0,1.0,9.0...|             18.0| 18.19844980179279|
|[1.0,43.0,0.0,6.0...|             16.0|16.200467711321494|
|[1.0,43.0,0.0,6.0...|             17.0|17.180630813682974|
|[1.0,43.0,0.0,6.0...|             18.0|17.180630813682974|
|[1.0,43.0,0.0,8.0...|             16.0| 16.95844849241083|
|[1.0,43.0,0.0,9.0...|             18.0|